In [328]:
#Chargement des données: train_set et test_set
import pandas as pd # importation du module pandas

test_set = pd.read_csv('test_set.csv')
train_set = pd.read_csv('train_set.csv')


# Devoir 2 - 8IAR403
Étudiant : *Mael Garnier*


Plan :
